In [2]:
# -*- coding: utf-8 -*-
from os.path import join
import codecs
import math
from collections import defaultdict as dd
from utils.embedding import EmbeddingModel
from utils.embedding import MySenteces1
from datetime import datetime
from utils.cache import LMDBClient
from utils import data_utils
from utils import settings
import re
import pandas as pd

start_time = datetime.now()

##将数据帧转化为列表字典，进一步变为二级字典
def data_todict(dataset):
    dataset_dict = dataset.to_dict(orient = 'records')
    data_dict = {}
    for pap in dataset_dict:
        pap['author'] = pap['author'].split('|')
        data_dict[pap['uid']] = pap 
    return data_dict

In [7]:
def dump_file_todict():
    ##采用分块读取的方法，主要用到参数chunksize，iterator参数（常用参数）
    yy = pd.read_csv('/home/wss/sites/disamb/sci_process/data/t_018_sci_disamb_string_precess.csv',
                       usecols = ['uid','author','title','abstract','keyword','org_name','pubyear','source'],sep = ',',iterator=True,encoding ='utf-8')

    # df = yy.get_chunk(1)
    # print(len(df))
    # print(df.columns)
    # print(df.head)
    loop = True
    chunkSize = 5000
    cnt = 0
    lc = LMDBClient('sci_all_data')
    while loop:
        try:
            chunk = yy.get_chunk(chunkSize)
            cnt += 1
            print('sci1800万论文存储了：%0.2f 万行'%(cnt*0.5))
            dataset_dict = chunk.to_dict(orient = 'records')
            for pap in dataset_dict:
                pap['author'] = pap['author'].split('|')
                pid_order = pap['uid']
                lc.set(pid_order, pap)
        except StopIteration:
            loop = False
            print("Iteration is stopped.")
    lc.db.close()
    print("分块处理存取进lmdb数据库用时为：%0.2f s"%(time.time()-start))

def extract_author_features(pub):
    stopword = ['at','based','in','of','for','on','and','to','an','with','the','by','this','we','be','is','are','can','or','no','from','like']
    org = ''
    if type(pub["org_name"]) is str:
        org = pub['org_name']
    if type(pub["title"]) is str:
        title = pub['title']
    else:
        title = ''
    if type(pub["abstract"]) is str:
        abstract = pub["abstract"]
    else:
        abstract = ''
    if type(pub["source"]) is str:
        source = pub['source']
    else:
        source = ''
    if 'pubyear' in pub and pub['pubyear'] is not None:
        year = str(pub['pubyear'])
    else:
        year = ''
    pstr = org+ ' '+title+' '+source+' '+abstract+' '+year
    feature = re.sub('[^a-z0-9A-Z]', ' ',pstr).strip().lower().split() 
    feature = [word for word in feature if word not in stopword and len(word)>2]   
    return feature

def dump_features_to_cache():
    '''
    generate author features by raw publication data and dump to cache
    
    '''
    lc = LMDBClient('sci_all_data')
    lm = LMDBClient('sci_all_data_feature')
    cnt = 0
    with lc.db.begin() as txn:
        for k in txn.cursor():
            cnt += 1
            pid = k[0].decode()
            paper = data_utils.deserialize_embedding(k[1])
            if len(paper["author"]) > 100:
                print(cnt, pid, len(paper["author"]))
                continue
            features = extract_author_features(paper)
            if cnt % 10000 == 0:
                print('已经提取：%d 万篇论文'%(cnt/10000))
            lm.set(pid,features)
    lm.db.close()
    lc.db.close()
            
            

def cal_feature_idf():
    """
    calculate word IDF (Inverse document frequency) using publication data
    """
    feature_dir = join(settings.DATA_DIR, 'global')
    counter = dd(int)
    cnt = 0
    LMDB_NAME = 'sci_all_data_feature'
    lc = LMDBClient(LMDB_NAME)
    author_cnt = 0
    with lc.db.begin() as txn:
        for k in txn.cursor():
#            print(k[0])
            features = data_utils.deserialize_embedding(k[1])
#            print(features)
            if author_cnt % 10000 == 0:
                print(author_cnt, features[0], counter.get(features[0]))
            author_cnt += 1
            for f in features:
                cnt += 1
                counter[f] += 1
    idf = {}
    for k in counter:
        idf[k] = math.log(cnt / counter[k])
    data_utils.dump_data(dict(idf), feature_dir, "feature_idf.pkl")
    
    
def dump_paper_embs():
    """
    dump author embedding to lmdb
    author embedding is calculated by weighted-average of word vectors with IDF
    """
    emb_model = EmbeddingModel.Instance()
    idf = data_utils.load_data(settings.GLOBAL_DATA_DIR, 'feature_idf.pkl')
    print('idf loaded')
    LMDB_NAME_FEATURE = 'sci_all_data_feature'
    lc_feature = LMDBClient(LMDB_NAME_FEATURE)
    LMDB_NAME_EMB = "sci_allpaper.emb.weighted"
    lc_emb = LMDBClient(LMDB_NAME_EMB)
    model = emb_model.load(emb_model.name)
    cnt = 0
    with lc_feature.db.begin() as txn:
        for k in txn.cursor():
            if cnt % 10000 == 0:
                print('cnt', cnt, datetime.now()-start_time)
            cnt += 1
            pid_order = k[0].decode('utf-8')
            features = data_utils.deserialize_embedding(k[1])
            cur_emb = emb_model.project_embedding(features, idf,model)
            if cur_emb is not None:
                lc_emb.set(pid_order, cur_emb)

In [25]:
dump_features_to_cache()

已经提取：1 万篇论文
13703 WOS:000208219100001 3742
已经提取：2 万篇论文
28186 WOS:000208475100001 170
已经提取：3 万篇论文
37665 WOS:000208625800019 164
已经提取：4 万篇论文
已经提取：5 万篇论文
已经提取：6 万篇论文
已经提取：7 万篇论文
已经提取：8 万篇论文
已经提取：9 万篇论文
已经提取：10 万篇论文
已经提取：11 万篇论文
已经提取：12 万篇论文
122615 WOS:000209134200006 440
已经提取：13 万篇论文
已经提取：14 万篇论文
已经提取：15 万篇论文
153713 WOS:000209415400010 114
157929 WOS:000209467300001 161
已经提取：16 万篇论文
已经提取：17 万篇论文
已经提取：18 万篇论文
已经提取：19 万篇论文
已经提取：20 万篇论文
已经提取：21 万篇论文
212357 WOS:000209772500001 266
已经提取：22 万篇论文
已经提取：23 万篇论文
已经提取：24 万篇论文
已经提取：25 万篇论文
258063 WOS:000209843500045 111
已经提取：26 万篇论文
269157 WOS:000209907000037 120
已经提取：27 万篇论文
277035 WOS:000213809900005 209
277876 WOS:000215723500001 120
已经提取：28 万篇论文
282863 WOS:000216599300024 198
已经提取：29 万篇论文
298286 WOS:000272669900001 355
已经提取：30 万篇论文
300138 WOS:000272790400077 277
302755 WOS:000272866500033 102
305101 WOS:000272912700003 360
305103 WOS:000272912700005 293
305104 WOS:000272912700006 1232
309375 WOS:000273021000024 256
309379 WOS:000273021000028 218


已经提取：81 万篇论文
814054 WOS:000278054000004 168
已经提取：82 万篇论文
824911 WOS:000278144800050 436
824912 WOS:000278144800051 424
824933 WOS:000278144800072 142
824935 WOS:000278144800074 161
824987 WOS:000278145100001 106
824988 WOS:000278145100002 155
824992 WOS:000278145100006 654
824993 WOS:000278145100007 543
824994 WOS:000278145100008 142
824995 WOS:000278145100009 102
825112 WOS:000278146700004 844
825998 WOS:000278150700009 105
已经提取：83 万篇论文
836918 WOS:000278242600007 154
837514 WOS:000278249000042 122
837554 WOS:000278250000023 351
837590 WOS:000278250000059 334
837742 WOS:000278251300016 292
已经提取：84 万篇论文
842057 WOS:000278301800001 101
846861 WOS:000278341300032 423
846862 WOS:000278341300033 149
已经提取：85 万篇论文
855379 WOS:000278429500036 102
857844 WOS:000278459000022 233
已经提取：86 万篇论文
861460 WOS:000278493500005 165
864073 WOS:000278522100003 102
864180 WOS:000278523400005 149
已经提取：87 万篇论文
871580 WOS:000278576500021 208
874530 WOS:000278592000016 137
876042 WOS:000278606600001 537
877884 WOS

1377563 WOS:000283051700002 180
1378970 WOS:000283064200002 603
1378972 WOS:000283064200004 210
1378973 WOS:000283064200005 152
1378974 WOS:000283064200006 122
1378975 WOS:000283064200007 104
1378978 WOS:000283064200010 105
1379015 WOS:000283064200047 220
已经提取：138 万篇论文
1385375 WOS:000283126900005 522
已经提取：139 万篇论文
1391007 WOS:000283211800005 262
1391009 WOS:000283211800007 132
1398249 WOS:000283294100021 112
1398526 WOS:000283297300032 212
已经提取：140 万篇论文
1402533 WOS:000283354100001 180
1403697 WOS:000283361800002 360
1403698 WOS:000283361800003 308
已经提取：141 万篇论文
1418141 WOS:000283490400001 171
已经提取：142 万篇论文
1422330 WOS:000283538000005 173
1422429 WOS:000283539300001 522
1422529 WOS:000283540500010 536
1422530 WOS:000283540500011 421
1422534 WOS:000283540500015 157
1422535 WOS:000283540500016 143
1423017 WOS:000283548600039 643
1426731 WOS:000283579000005 105
已经提取：143 万篇论文
1434923 WOS:000283647800009 106
1434928 WOS:000283647800014 159
1434929 WOS:000283647800015 111
1435062 WOS:00028364

1897656 WOS:000287850600001 263
1897664 WOS:000287850600009 260
已经提取：190 万篇论文
1905703 WOS:000287928600003 126
1906708 WOS:000287937700078 2489
1906709 WOS:000287937900001 2477
1906927 WOS:000287939200001 360
1908937 WOS:000287955500002 249
1908939 WOS:000287955500004 189
1909380 WOS:000287960700001 160
1909420 WOS:000287961100001 468
已经提取：191 万篇论文
1918781 WOS:000288041900003 354
1918782 WOS:000288041900004 103
已经提取：192 万篇论文
1926570 WOS:000288121000001 494
1926571 WOS:000288121000002 3605
已经提取：193 万篇论文
1933529 WOS:000288163100011 418
1936668 WOS:000288184800003 147
已经提取：194 万篇论文
1949699 WOS:000288300400006 3332
已经提取：195 万篇论文
1951820 WOS:000288319800001 180
1951838 WOS:000288320200005 574
已经提取：196 万篇论文
1965015 WOS:000288441900041 239
1965654 WOS:000288448700001 476
1965656 WOS:000288448700003 185
已经提取：197 万篇论文
1973783 WOS:000288511800004 186
1974287 WOS:000288515000004 2502
1976354 WOS:000288533200001 3642
1977130 WOS:000288541600091 238
1978302 WOS:000288550800006 160
1978417 WOS:000288

2475930 WOS:000292713200007 144
2477127 WOS:000292719700001 3589
2477128 WOS:000292719700002 252
2477129 WOS:000292719700003 3285
2477130 WOS:000292719700004 3307
2478560 WOS:000292736700008 461
已经提取：248 万篇论文
2481467 WOS:000292767200010 158
2481529 WOS:000292768900001 520
2489009 WOS:000292822700003 495
已经提取：249 万篇论文
2494502 WOS:000292875300002 133
2499210 WOS:000292922100003 3602
已经提取：250 万篇论文
2502817 WOS:000292952900001 3622
2504466 WOS:000292969900001 417
2504672 WOS:000292971400001 126
2505289 WOS:000292977400030 252
2505315 WOS:000292977400056 503
2506606 WOS:000292990000001 385
2506754 WOS:000292993900003 2408
2506756 WOS:000292993900005 3607
2506757 WOS:000292993900006 2407
2506785 WOS:000292994100005 3322
2508997 WOS:000293017700080 243
2509101 WOS:000293017700184 242
2509111 WOS:000293017700194 143
2509899 WOS:000293027100016 203
2509900 WOS:000293027100017 250
已经提取：251 万篇论文
已经提取：252 万篇论文
2520795 WOS:000293125400008 216
2521356 WOS:000293130500001 516
2521364 WOS:0002931306000

...

17492414 WOS:000475510900004 1783
17492415 WOS:000475510900005 108
17494535 WOS:000475553300016 3214
17494581 WOS:000475554500004 461
17495311 WOS:000475572300001 1773
17495321 WOS:000475573300001 1778
17495860 WOS:000475591000003 228
17496430 WOS:000475616600001 108
17496485 WOS:000475618400001 121
17497558 WOS:000475657100005 156
17497728 WOS:000475662300001 171
17497775 WOS:000475664200011 116
已经提取：1750 万篇论文
17500383 WOS:000475686000001 161
17500793 WOS:000475689500010 303
17502113 WOS:000475702100012 111
17502142 WOS:000475702500002 160
17503640 WOS:000475730300002 112
17504323 WOS:000475741400001 391
17504727 WOS:000475747400002 122
17505945 WOS:000475776000010 178
17505990 WOS:000475778100002 945
17506200 WOS:000475783500012 120
17506335 WOS:000475787700005 200
17506381 WOS:000475788200020 109
17507162 WOS:000475814500002 101
17507767 WOS:000475832500026 704
已经提取：1751 万篇论文
17513358 WOS:000475846300004 148
17513363 WOS:000475846300009 229
17513464 WOS:000475849000004 133
17515729 

17616056 WOS:000478679900045 1276
17616165 WOS:000478681800012 186
17616621 WOS:000478688200026 285
17617115 WOS:000478698300025 114
17617812 WOS:000478707300003 110
17619689 WOS:000478730300004 245
17619771 WOS:000478732300001 188
已经提取：1762 万篇论文
17621457 WOS:000478735300006 139
17622025 WOS:000478743700055 109
17622486 WOS:000478754100005 564
17623612 WOS:000478770300009 123
17623927 WOS:000478778000031 720
17623940 WOS:000478778000044 144
17624029 WOS:000478778600021 460
17624284 WOS:000478784800019 120
17624302 WOS:000478784800037 133
17624304 WOS:000478784800039 196
17624305 WOS:000478784800040 190
17624516 WOS:000478787500006 1285
17624539 WOS:000478787900019 340
17624547 WOS:000478787900027 262
17625148 WOS:000478794700022 172
17625766 WOS:000478807400003 116
17626891 WOS:000478838200038 124
17626899 WOS:000478838200046 221
17626901 WOS:000478838200048 707
17626906 WOS:000478838200053 222
17626909 WOS:000478838200056 211
17626924 WOS:000478838200071 106
17626977 WOS:0004788400000

17758847 WOS:000482247500019 161
17759242 WOS:000482248600008 124
已经提取：1776 万篇论文
17761343 WOS:000482303700001 113
17761359 WOS:000482304300001 339
17761362 WOS:000482304300004 204
17761639 WOS:000482319700001 177
17761696 WOS:000482319700058 143
17761750 WOS:000482320300001 138
17762236 WOS:000482332500076 136
17762449 WOS:000482340700007 224
17762518 WOS:000482340700076 115
17762651 WOS:000482350100003 186
17762726 WOS:000482354100014 149
17762757 WOS:000482356100007 157
17762802 WOS:000482357500004 163
17762808 WOS:000482357500010 151
17762809 WOS:000482357500011 161
17762956 WOS:000482366600001 108
17763004 WOS:000482373200005 145
17763007 WOS:000482373200008 145
17763021 WOS:000482373600006 183
17763023 WOS:000482373600008 125
17763030 WOS:000482373600015 140
17764365 WOS:000482401700001 1889
17764850 WOS:000482411700002 155
17765213 WOS:000482420100006 150
17765406 WOS:000482423400010 191
17765688 WOS:000482433800011 116
17765691 WOS:000482433800014 168
17766044 WOS:00048243750001

17877299 WOS:000485087400011 298
17879080 WOS:000485140300046 166
17879721 WOS:000485158400026 230
17879799 WOS:000485158400104 147
17879885 WOS:000485163700019 108
已经提取：1788 万篇论文
17880103 WOS:000485168600025 202
17880146 WOS:000485169900020 272
17880184 WOS:000485171100012 348
17880186 WOS:000485171100014 146
17880363 WOS:000485185200001 233
17880457 WOS:000485187800001 284
17880859 WOS:000485195900003 6973
17880870 WOS:000485196300001 123
17880878 WOS:000485196600001 269
17880879 WOS:000485196600002 4910
17880891 WOS:000485197000002 419
17880892 WOS:000485197000003 713
17880904 WOS:000485197200001 692
17880925 WOS:000485197400001 403
17881094 WOS:000485202800003 170
17882288 WOS:000485215700003 130
17882400 WOS:000485216900016 402
17882435 WOS:000485217300010 197
17883500 WOS:000485244600001 196
17884762 WOS:000485269700075 206
17884767 WOS:000485269700080 1097
17884777 WOS:000485269700090 170
17884779 WOS:000485269700092 103
17884800 WOS:000485269700113 162
17884812 WOS:000485269700

18038459 WOS:000488838600040 306
18038464 WOS:000488838600045 201
18038465 WOS:000488838600046 120
18038668 WOS:000488842600015 166
18038986 WOS:000488847800030 230
已经提取：1804 万篇论文
18043920 WOS:000488949500009 204
18044234 WOS:000488953400017 118
18048507 WOS:000489016400010 247
18049241 WOS:000489035200014 161
已经提取：1805 万篇论文
18050704 WOS:000489065200002 191
18052579 WOS:000489101100002 194
18058035 WOS:000489163100052 103
18059781 WOS:000489184200071 338
已经提取：1806 万篇论文
18062993 WOS:000489251000001 2466
18062994 WOS:000489251000002 2402
18063144 WOS:000489257500032 183
18065311 WOS:000489292800018 164
已经提取：1807 万篇论文
18071329 WOS:000489332200017 109
18071783 WOS:000489337300052 140
18072251 WOS:000489353300005 139
18074174 WOS:000489524200013 133
18075185 WOS:000489558300002 119
18075217 WOS:000489560800001 157
18075708 WOS:000489575100004 397
18075720 WOS:000489575100016 191
18075859 WOS:000489577600005 1075
18075885 WOS:000489577700025 104
18076679 WOS:000489613700030 118
已经提取：1808 万篇论

In [8]:
dump_paper_embs()

idf loaded
cnt 0 0:11:25.127591
cnt 10000 0:11:42.383320
cnt 20000 0:11:59.791335
cnt 30000 0:12:20.053584
cnt 40000 0:12:41.223218
cnt 50000 0:12:59.704242
cnt 60000 0:13:19.835133
cnt 70000 0:13:37.617313
cnt 80000 0:13:56.585290
cnt 90000 0:14:15.676082
cnt 100000 0:14:34.565754
cnt 110000 0:14:53.865560
cnt 120000 0:15:14.024566
cnt 130000 0:15:34.411814
cnt 140000 0:15:58.925874
cnt 150000 0:16:16.076748
cnt 160000 0:16:37.748341
cnt 170000 0:16:55.532723
cnt 180000 0:17:16.494184
cnt 190000 0:17:36.984642
cnt 200000 0:17:54.876865
cnt 210000 0:18:14.120904
cnt 220000 0:18:31.672736
cnt 230000 0:18:48.930344
cnt 240000 0:19:07.312269
cnt 250000 0:19:24.734871
cnt 260000 0:19:43.775096
cnt 270000 0:20:03.212434
cnt 280000 0:20:22.816520
cnt 290000 0:20:45.954007
cnt 300000 0:21:10.886041
cnt 310000 0:21:36.260046
cnt 320000 0:22:01.317212
cnt 330000 0:22:25.878544
cnt 340000 0:22:50.452941
cnt 350000 0:23:14.247808
cnt 360000 0:23:38.982743
cnt 370000 0:24:05.194900
cnt 380000 0:24

cnt 3080000 2:13:31.297797
cnt 3090000 2:13:56.568228
cnt 3100000 2:14:20.960034
cnt 3110000 2:14:46.199520
cnt 3120000 2:15:13.471462
cnt 3130000 2:15:38.184724
cnt 3140000 2:16:02.598260
cnt 3150000 2:16:27.525783
cnt 3160000 2:16:52.593076
cnt 3170000 2:17:16.746959
cnt 3180000 2:17:42.194440
cnt 3190000 2:18:07.708499
cnt 3200000 2:18:33.328956
cnt 3210000 2:18:58.714264
cnt 3220000 2:19:24.704668
cnt 3230000 2:19:47.979856
cnt 3240000 2:20:15.159527
cnt 3250000 2:20:39.818835
cnt 3260000 2:20:59.164645
cnt 3270000 2:21:23.898248
cnt 3280000 2:21:48.212255
cnt 3290000 2:22:10.756838
cnt 3300000 2:22:32.874040
cnt 3310000 2:22:54.467700
cnt 3320000 2:23:17.799621
cnt 3330000 2:23:41.224550
cnt 3340000 2:24:08.124327
cnt 3350000 2:24:31.650270
cnt 3360000 2:24:56.411035
cnt 3370000 2:25:22.985679
cnt 3380000 2:25:48.030398
cnt 3390000 2:26:11.791139
cnt 3400000 2:26:36.052264
cnt 3410000 2:27:01.015783
cnt 3420000 2:27:25.463310
cnt 3430000 2:27:48.940946
cnt 3440000 2:28:13.335232
c

cnt 6120000 4:17:51.755611
cnt 6130000 4:18:16.090700
cnt 6140000 4:18:39.870767
cnt 6150000 4:19:05.203428
cnt 6160000 4:19:29.375876
cnt 6170000 4:19:53.902970
cnt 6180000 4:20:18.678198
cnt 6190000 4:20:44.997534
cnt 6200000 4:21:09.859996
cnt 6210000 4:21:34.421830
cnt 6220000 4:21:56.440671
cnt 6230000 4:22:21.228796
cnt 6240000 4:22:45.226420
cnt 6250000 4:23:09.888144
cnt 6260000 4:23:35.097732
cnt 6270000 4:23:59.672185
cnt 6280000 4:24:24.430499
cnt 6290000 4:24:49.627808
cnt 6300000 4:25:14.556440
cnt 6310000 4:25:36.295422
cnt 6320000 4:26:02.441960
cnt 6330000 4:26:26.699755
cnt 6340000 4:26:50.866917
cnt 6350000 4:27:15.524595
cnt 6360000 4:27:38.415029
cnt 6370000 4:28:02.626056
cnt 6380000 4:28:26.850582
cnt 6390000 4:28:52.138071
cnt 6400000 4:29:17.256476
cnt 6410000 4:29:40.988177
cnt 6420000 4:30:05.622330
cnt 6430000 4:30:30.259398
cnt 6440000 4:30:55.178409
cnt 6450000 4:31:20.374429
cnt 6460000 4:31:44.635603
cnt 6470000 4:32:09.741666
cnt 6480000 4:32:34.277380
c

cnt 9160000 6:23:25.737120
cnt 9170000 6:23:49.069007
cnt 9180000 6:24:17.347560
cnt 9190000 6:24:43.302511
cnt 9200000 6:25:08.766499
cnt 9210000 6:25:34.221854
cnt 9220000 6:25:59.646774
cnt 9230000 6:26:27.172560
cnt 9240000 6:26:53.025185
cnt 9250000 6:27:15.313425
cnt 9260000 6:27:41.674375
cnt 9270000 6:28:06.753944
cnt 9280000 6:28:31.534208
cnt 9290000 6:28:57.111642
cnt 9300000 6:29:22.141917
cnt 9310000 6:29:44.726191
cnt 9320000 6:30:09.207174
cnt 9330000 6:30:35.223959
cnt 9340000 6:31:01.981035
cnt 9350000 6:31:27.800157
cnt 9360000 6:31:53.610884
cnt 9370000 6:32:17.187486
cnt 9380000 6:32:39.511191
cnt 9390000 6:33:00.471604
cnt 9400000 6:33:21.957743
cnt 9410000 6:33:44.674716
cnt 9420000 6:34:07.677059
cnt 9430000 6:34:32.591150
cnt 9440000 6:34:55.777780
cnt 9450000 6:35:19.865282
cnt 9460000 6:35:41.450406
cnt 9470000 6:36:04.774214
cnt 9480000 6:36:31.018918
cnt 9490000 6:36:55.413584
cnt 9500000 6:37:20.601120
cnt 9510000 6:37:45.160342
cnt 9520000 6:38:09.517774
c

cnt 12120000 8:27:11.017549
cnt 12130000 8:27:36.778897
cnt 12140000 8:28:02.919536
cnt 12150000 8:28:27.765860
cnt 12160000 8:28:52.717895
cnt 12170000 8:29:16.955621
cnt 12180000 8:29:42.458897
cnt 12190000 8:30:08.288917
cnt 12200000 8:30:31.265033
cnt 12210000 8:30:57.868274
cnt 12220000 8:31:21.932397
cnt 12230000 8:31:44.292876
cnt 12240000 8:32:12.285003
cnt 12250000 8:32:37.822730
cnt 12260000 8:33:02.080436
cnt 12270000 8:33:27.363843
cnt 12280000 8:33:53.997348
cnt 12290000 8:34:19.433774
cnt 12300000 8:34:46.416791
cnt 12310000 8:35:13.116947
cnt 12320000 8:35:36.612208
cnt 12330000 8:36:02.463306
cnt 12340000 8:36:29.637014
cnt 12350000 8:36:54.831166
cnt 12360000 8:37:22.183770
cnt 12370000 8:37:47.673757
cnt 12380000 8:38:11.343625
cnt 12390000 8:38:35.434396
cnt 12400000 8:39:01.055409
cnt 12410000 8:39:26.919696
cnt 12420000 8:39:52.960131
cnt 12430000 8:40:19.008916
cnt 12440000 8:40:42.710753
cnt 12450000 8:41:10.157558
cnt 12460000 8:41:36.611311
cnt 12470000 8:42:05

cnt 15020000 10:32:57.901342
cnt 15030000 10:33:24.179095
cnt 15040000 10:33:53.292750
cnt 15050000 10:34:19.996501
cnt 15060000 10:34:47.686467
cnt 15070000 10:35:07.742841
cnt 15080000 10:35:33.482259
cnt 15090000 10:36:00.122867
cnt 15100000 10:36:28.052649
cnt 15110000 10:36:55.260670
cnt 15120000 10:37:23.055601
cnt 15130000 10:37:51.936896
cnt 15140000 10:38:20.728748
cnt 15150000 10:38:47.880199
cnt 15160000 10:39:11.843173
cnt 15170000 10:39:31.382188
cnt 15180000 10:39:59.022017
cnt 15190000 10:40:28.759834
cnt 15200000 10:40:57.851886
cnt 15210000 10:41:23.025601
cnt 15220000 10:41:51.873092
cnt 15230000 10:42:20.150458
cnt 15240000 10:42:49.375893
cnt 15250000 10:43:16.690159
cnt 15260000 10:43:41.221339
cnt 15270000 10:44:09.059784
cnt 15280000 10:44:37.726716
cnt 15290000 10:45:04.336988
cnt 15300000 10:45:32.958528
cnt 15310000 10:46:02.610097
cnt 15320000 10:46:30.238248
cnt 15330000 10:46:58.659099
cnt 15340000 10:47:22.837265
cnt 15350000 10:47:46.143847
cnt 15360000 1

cnt 17850000 12:53:14.038693
cnt 17860000 12:53:49.287730
cnt 17870000 12:54:23.367481
cnt 17880000 12:54:50.873760
cnt 17890000 12:55:19.478295
cnt 17900000 12:55:51.274949
cnt 17910000 12:56:23.001148
cnt 17920000 12:56:55.930446
cnt 17930000 12:57:26.627928
cnt 17940000 12:57:59.775271
cnt 17950000 12:58:28.488773
cnt 17960000 12:58:58.561359
cnt 17970000 12:59:26.145281
cnt 17980000 12:59:56.818528
cnt 17990000 13:00:21.954610
cnt 18000000 13:00:45.765098
cnt 18010000 13:01:11.751350
cnt 18020000 13:01:33.077141
cnt 18030000 13:01:55.665396
cnt 18040000 13:02:21.391291
cnt 18050000 13:02:46.817014
cnt 18060000 13:03:10.160016
cnt 18070000 13:03:38.019987
cnt 18080000 13:04:01.366062
cnt 18090000 13:04:27.499740
cnt 18100000 13:04:56.022575
cnt 18110000 13:05:21.475558
cnt 18120000 13:05:46.021888


In [26]:
lc = LMDBClient('sci_all_data')
feature1 = lc.get('WOS:000208475100001')
# print(feature1)
# print(lc.get('WOS:000430230400007'))
lc.db.close()